In [1]:
import spacy
import re
import sys
import random
sys.path.append("..")

from datasets.newsgroups import NewsgroupsDataset
from train import get_args

In [5]:
"""
Load dataset with custom arguments
Note: May take a couple mins
"""

class args:
    def __init__(self):
        default_args = get_args()
        
        self.dataset_dir = '../data/'
        self.toy = False
        self.file_batch_size = default_args.file_batch_size
        self.window_size = default_args.window_size
        
dataset = NewsgroupsDataset(args())

  0%|          | 0/46 [00:00<?, ?it/s]


Using cleaned newsgroups data from scklearn...

Generating Examples for Dataset (multi-threaded)...


100%|██████████| 46/46 [04:28<00:00,  2.37s/it]


In [31]:
"""
View a random file's tokenization
"""

rand_doc = random.choice(dataset.files)
tokenized_doc = dataset.tokenizer.tokenize_doc(rand_doc)

print(f'Tokenized Document:\n\n{tokenized_doc}')
print(f'\nRandom Document:\n\n{rand_doc}')

Tokenized Document:

['andersomspot', 'colorado', 'edu', 'marcanderson', 'methods', 'year', 'crime', 'bill', 'teeth', 'bare', 'gums', 'clinton', 'said', 'particular', 'administration', 'place', 'strict', 'controls', 'data', 'formats', 'protocols', 'require', 'registration', 'called', 'cryptographic', 'keys', 'hope', 'denying', 'drug', 'dealers', 'ability', 'communicate', 'secret', 'clinton', 'said', 'approach', 'crackdowns', 'forms', 'underground', 'economic', 'activity', 'deficit', 'causing', 'tax', 'evaders', 'live', 'luxury', 'expense', 'grandchildren', 'people', 'thought', 'exaggerating', 'claiming', 'cripplechip', 'step', 'totalitarian', 'plot', 'civil', 'liberties', 'usa', 'optimist', 'things', 'happening', 'faster', 'expected', 'dirty', 'tricks', 'apply', 'communist', 'regime', 'secret', 'tell', 'people', 'fact', 'course', 'explaining', 'better', 'situation', 'previous', 'messages', 'wrote', 'americans', 'wake', 'fight', 'new', 'proposal', 'late', 'happened', 'civil', 'liberties